# About this notebook

The purpose of this notebook is to upload previously acquired gridMet data to Amazon s3. Before running this notebook, you should have previously run notebook 1 in this directory in order to download gridMet data locally, potentially changing the range of years, in order to acquire data for the desired date range.

In [1]:
import boto3
import pandas as pd
import os
import re
import numpy as np
from pprint import pprint

In [2]:
gridMet_yearly_data = [f for f in os.listdir('.') if re.match(r'gridMet_\d{4}.parquet.gz', f)]
pprint(gridMet_yearly_data)

['gridMet_1999.parquet.gz',
 'gridMet_2000.parquet.gz',
 'gridMet_2001.parquet.gz',
 'gridMet_2002.parquet.gz',
 'gridMet_2003.parquet.gz',
 'gridMet_2004.parquet.gz',
 'gridMet_2005.parquet.gz',
 'gridMet_2006.parquet.gz',
 'gridMet_2007.parquet.gz',
 'gridMet_2008.parquet.gz',
 'gridMet_2009.parquet.gz',
 'gridMet_2010.parquet.gz',
 'gridMet_2011.parquet.gz',
 'gridMet_2012.parquet.gz',
 'gridMet_2013.parquet.gz',
 'gridMet_2014.parquet.gz',
 'gridMet_2015.parquet.gz',
 'gridMet_2016.parquet.gz',
 'gridMet_2017.parquet.gz',
 'gridMet_2018.parquet.gz',
 'gridMet_2019.parquet.gz']


In [3]:
%%time
df = pd.DataFrame()
for file in gridMet_yearly_data:
    df = pd.concat([df, pd.read_parquet(file)])
print(df.shape)

(10676640, 14)
Wall time: 17.4 s


In [4]:
df.head()

precipitation_amount_mm  \
date       latitude  longitude                              
1999-01-01 33.566667 -117.975000                      0.0   
                     -117.933333                      0.0   
                     -117.891667                      0.0   
                     -117.850000                      0.0   
                     -117.808333                      0.0   

                                  relative_humidity_%  \
date       latitude  longitude                          
1999-01-01 33.566667 -117.975000                 40.3   
                     -117.933333                 39.8   
                     -117.891667                 38.2   
                     -117.850000                 36.4   
                     -117.808333                 33.8   

                                  specific_humidity_kg/kg  \
date       latitude  longitude                              
1999-01-01 33.566667 -117.975000                  0.00589   
                     -117.933333                  0.00590   
                     -117.891667                  0.00580   
                     -117.850000                  0.00567   
                     -117.808333                  0.00538   

                                  surface_downwelling_shortwave_flux_in_air_W m-2  \
date       latitude  longitude                                                      
1999-01-01 33.566667 -117.975000                                            138.0   
                     -117.933333                                            137.2   
                     -117.891667                                            137.2   
                     -117.850000                                            137.3   
                     -117.808333                                            137.3   

                                  wind_from_direction_Degrees Clockwise from north  \
date       latitude  longitude                                                       
1999-01-01 33.566667 -117.975000                                             123.0   
                     -117.933333                                             123.0   
                     -117.891667                                             123.0   
                     -117.850000                                              49.0   
                     -117.808333                                              49.0   

                                  wind_speed_m/s  max_air_temperature_K  \
date       latitude  longitude                                            
1999-01-01 33.566667 -117.975000             1.6                  293.1   
                     -117.933333             1.6                  293.1   
                     -117.891667             1.7                  293.2   
                     -117.850000             1.8                  293.3   
                     -117.808333             1.9                  293.2   

                                  min_air_temperature_K  \
date       latitude  longitude                            
1999-01-01 33.566667 -117.975000                  281.1   
                     -117.933333                  281.2   
                     -117.891667                  281.2   
                     -117.850000                  280.3   
                     -117.808333                  279.9   

                                  burning_index_g_Unitless  \
date       latitude  longitude                               
1999-01-01 33.566667 -117.975000                      24.0   
                     -117.933333                      24.0   
                     -117.891667                      26.0   
                     -117.850000                      27.0   
                     -117.808333                      29.0   

                                  dead_fuel_moisture_100hr_Percent  \
date       latitude  longitude                                       
1999-01-01 33.566667 -117.975000                              16

In [5]:
df.describe()

,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa
count,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06,6.826300e+06
mean,8.304122e-01,3.000426e+01,6.503330e-03,2.385361e+02,2.266884e+02,2.980093e+00,2.976084e+02,2.834156e+02,3.714322e+01,1.201792e+01,1.275536e+01,5.026444e+01,4.325072e+00,1.354994e+00
std,4.286879e+00,1.769667e+01,2.908315e-03,7.989067e+01,7.785217e+01,1.255808e+00,7.771557e+00,6.103147e+00,1.910992e+01,4.536604e+00,4.035236e+00,2.096610e+01,1.928999e+00,9.730105e-01
min,0.000000e+00,1.000000e-01,2.400000e-04,4.500000e+00,0.000000e+00,4.000000e-01,2.686000e+02,2.575000e+02,0.000000e+00,1.900000e+00,3.400000e+00,0.000000e+00,2.000000e-01,0.000000e+00
25%,0.000000e+00,1.540000e+01,4.310000e-03,1.649000e+02,1.970000e+02,2.200000e+00,2.921000e+02,2.791000e+02,2.700000e+01,8.100000e+00,9.600000e+00,3.400000e+01,2.700000e+00,6.300000e-01
50%,0.000000e+00,2.670000e+01,6.200000e-03,2.456000e+02,2.410000e+02,2.700000e+00,2.970000e+02,2.833000e+02,3.700000e+01,1.210000e+01,1.290000e+01,4.700000e+01,4.200000e+00,1.090000e+00
75%,0.000000e+00,4.260000e+01,8.430000e-03,3.129000e+02,2.770000e+02,3.500000e+00,3.031000e+02,2.879000e+02,5.000000e+01,1.540000e+01,1.560000e+01,6.600000e+01,5.700000e+00,1.830000e+00
max,2.871000e+02,1.000000e+02,2.392000e-02,3.777000e+02,3.600000e+02,1.620000e+01,3.227000e+02,3.074000e+02,1.510000e+02,3.060000e+01,3.080000e+01,1.100000e+02,1.320000e+01,7.170000e+00


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10676640 entries, (Timestamp('1999-01-01 00:00:00'), 33.56666666666667, -117.97499996666667) to (Timestamp('2019-12-31 00:00:00'), 32.400000000000006, -116.01666663333334)
Data columns (total 14 columns):
 #   Column                                            Dtype  
---  ------                                            -----  
 0   precipitation_amount_mm                           float64
 1   relative_humidity_%                               float64
 2   specific_humidity_kg/kg                           float64
 3   surface_downwelling_shortwave_flux_in_air_W m-2   float64
 4   wind_from_direction_Degrees Clockwise from north  float64
 5   wind_speed_m/s                                    float64
 6   max_air_temperature_K                             float64
 7   min_air_temperature_K                             float64
 8   burning_index_g_Unitless                          float64
 9   dead_fuel_moisture_100hr_Percent               

In [7]:
np.sum(df.isna())

precipitation_amount_mm                             3850340
relative_humidity_%                                 3850340
specific_humidity_kg/kg                             3850340
surface_downwelling_shortwave_flux_in_air_W m-2     3850340
wind_from_direction_Degrees Clockwise from north    3850340
wind_speed_m/s                                      3850340
max_air_temperature_K                               3850340
min_air_temperature_K                               3850340
burning_index_g_Unitless                            3850340
dead_fuel_moisture_100hr_Percent                    3850340
dead_fuel_moisture_1000hr_Percent                   3850340
energy_release_component-g_Unitless                 3850340
potential_evapotranspiration_mm                     3850340
mean_vapor_pressure_deficit_kPa                     3850340
dtype: int64

In [9]:
s3_url = 'dse-cohort5-group5'
s3 = boto3.client("s3")
all_objects = s3.list_objects(Bucket=s3_url)
all_objects

{'ResponseMetadata': {'RequestId': '92E5E72DFE2F2E58',
  'HostId': 'pJR7T7b5yeHrmuCSyvyYDuDOpDXf4i1o2nuEM9VXAEpEMTqaEASTFPZwWlTgnvldcb4vfR0KWoQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'pJR7T7b5yeHrmuCSyvyYDuDOpDXf4i1o2nuEM9VXAEpEMTqaEASTFPZwWlTgnvldcb4vfR0KWoQ=',
   'x-amz-request-id': '92E5E72DFE2F2E58',
   'date': 'Wed, 11 Mar 2020 04:39:13 GMT',
   'x-amz-bucket-region': 'us-west-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'test/',
   'LastModified': datetime.datetime(2020, 2, 14, 2, 26, 50, tzinfo=tzutc()),
   'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'Size': 0,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'kcoakley+cohort5group5',
    'ID': 'fe3a1755292b139ad5397ba537aa0aff32ef6ebba07fe45dcbe486d81e1ccc11'}},
  {'Key': 'wildfire_capstone/gridMet.parquet.gz',
   'LastModified': datetime.datetime(2020, 

In [10]:
s3_url = "s3://dse-cohort5-group5/wildfire_capstone/gridMet.parquet.gz"
df.to_parquet(s3_url, compression="gzip")